# A String Matching Story

### 1. Imports

In [1]:
import itertools
import numpy as np
import pandas as pd

from nltk.metrics.distance import edit_distance
from fuzzywuzzy import fuzz
from polyfuzz import PolyFuzz
from polyfuzz.models import TFIDF

C:\Users\ezequ\anaconda3\envs\bnb\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### 2. The data

In [2]:
def data_generator(variant,flavour,product,quantity):
    permutations=itertools.product(variant,flavour,product,quantity)
    combinations=list(permutations)
    data=pd.DataFrame(combinations, columns=["variant","flavour","product","quantity"])
    data["product_option_1"]="beverages "+data["product"]+" "+data["variant"]+" "+data["flavour"]+" "+data["quantity"]
    data["product_option_2"]="beverage "+data["variant"]+" "+data["flavour"]+" "+data["product"]+" "+data["quantity"]
    results=pd.concat([data["product_option_1"],data["product_option_2"]])
    results.drop_duplicates(inplace=True)
    results=pd.DataFrame(results,columns=["product"])
    return(results)

In [3]:
external_data=data_generator(variant=["diet","dit","diiet",""],
                             flavour=["vanilla","chocolate","acid"],
                             product=["coke","cokke","spriite","sprite","prite","fanta","afnta","pepsi"],
                             quantity=["500ml","1.5L","2.25L","354ml","500cc","354cc",
                                       "500 ml","1.5 L","2.25 L","354 ml","500 cc","354 cc",
                                       "500 ml.","1.5 LT.","2.25 L","354 ml","500 cc","354 cc",
                                       "500 ml.","1.5 l.","2.25 l","354 ml.","500 cc.","354 cc",
                                       "500 ml","1.5 lts","2.25 lts","354 ml","500 cc","354 cc",
                                       "500 ml.","1.5 lts.","2.25 lts","354 ml","500 cc","354 cc",
                                       "500 ml.","1.5 lt.","2.25 lt","354 ml.","500 cc.","354 cc"])

In [4]:
# Beverages of internal source
variant=["DIET"]
product=["COKE","SPRITE","FANTA","PEPSI"]
flavour=["VANILLA","CHOCOLATE","COCO","STRAWBERRY"]
quantity=["X 500ML","X 1.5L","X 2.25L","X 354ML"]
permutations=itertools.product(variant,flavour,product,quantity)
combinations=list(permutations)
internal_data=pd.DataFrame(combinations, columns=["variant","flavour","product","quantity"])
internal_data["product_name"]=internal_data["variant"]+" "+internal_data["flavour"]+" "+internal_data["product"]+" "+internal_data["quantity"]
internal_data=pd.DataFrame(internal_data["product_name"])

### 3. String preprocessing

In [5]:
def string_processing(df:pd.DataFrame,columns_to_clean:list):
    """
    Function to clean specific columns of a dataframe
    args:
        df= pandas dataframe
        ignored_columns=
    """
    for col in df.columns.difference(columns_to_clean):
        
        #Convert strings to lowercase
        df[col] = df[col].str.lower()
        
        #Remove the "-" symbol
        df[col] = df[col].str.replace(r"(\-)","",regex=True)

        # Remove words that don't add information
        df[col] = df[col].str.replace(r"(beverages|beverage)","",regex=True)
        
        # Replace consecutive blank spaces for 1 blank space
        df[col] = df[col].str.replace(r"(  |   )", " ",regex=True)
        df[col] = df[col].str.replace(r"(  )", " ",regex=True)
        
        # Remove starting blank space
        df[col] = df[col].str.replace(r"(^ )", "",regex=True)
        
        # Remove blank space at the end
        df[col] = df[col].str.replace(r"( $)", "",regex=True)
        
        # Standardize the spelling of litres and ml
        df[col] = df[col].str.replace(r"(\s?l$|\s?lt$|\s?lt\.$|\sl\.$|\s?lts$|\s?lts\.$)", "l",regex=True)
        df[col] = df[col].str.replace(r"((?<![\w])|(?<=\d))(ml\.?$|cc\.?$)", "ml",regex=True)
        df[col] = df[col].str.replace(r"(\s?ml$)", "ml",regex=True)
        df[col] = df[col].str.replace(r"((?<=\d))(\s?l$)", "l",regex=True)

In [6]:
# Create copies of main columns to be modified
external_data["product_cleaned_from"]=external_data["product"].copy()
internal_data["product_cleaned_to"]=internal_data["product_name"].copy()

In [7]:
# Clean columns
string_processing(df=external_data,columns_to_clean=["product"])
string_processing(df=internal_data,columns_to_clean=["product_name"])

### 4. String matching with Tf-Idf Vectorization

In [8]:
# We import sklearns' TFIDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
# Function to partition strings into n-grams (modified for the example)
def _create_ngrams(string: str):
        """ Create n_grams from a string
        Steps:
            * Extract character-level ngrams with `self.n_gram_range` (both ends inclusive)
            * Remove n-grams that have a whitespace in them
        """
        result = []
        for n in range(3, 4):
            ngrams = zip(*[string[i:] for i in range(n)])
            ngrams = [''.join(ngram) for ngram in ngrams if ' ' not in ngram]
            result.extend(ngrams)

        return result

In [10]:
# We define 2 lists to be matched, where the "to_list" can be seen as the reference (for example, a product catalogue)
from_list = ["coke 1.5L","sprite zero 1.5L"]
to_list = ["diet coke 1.5L"]

# We define the vectorizer using the union of elements of both lists
  # With min_df=1
  # With analyzer=_create_ngrams we are specifying the function to create the n grams
vectorizer = TfidfVectorizer(min_df=1, analyzer=_create_ngrams).fit(to_list + from_list)

# Ya teniendo el vectorizer generamos matrices esparsas con tantas columnas como n-gramas únicos y filas como palabras en cada lista
# After fitting the vectorizer, we generate sparse matrix with
 # as many columns as unique n-grams
 # as many rows as words in each list
tf_idf_to = vectorizer.transform(to_list)
tf_idf_from = vectorizer.transform(from_list)

In [11]:
# Check the vocabulary of the vectorizer, i.e. the unique partitions of n-grams
vectorizer.vocabulary_

{'die': 3,
 'iet': 5,
 'cok': 2,
 'oke': 7,
 '1.5': 1,
 '.5L': 0,
 'spr': 10,
 'pri': 8,
 'rit': 9,
 'ite': 6,
 'zer': 11,
 'ero': 4}

In [12]:
#We check the matrices
matrix_from=pd.DataFrame(tf_idf_from.todense(),columns=sorted(vectorizer.vocabulary_))
matrix_to=pd.DataFrame(tf_idf_to.todense(),columns=sorted(vectorizer.vocabulary_))

In [13]:
matrix_from

,.5L,1.5,cok,die,ero,iet,ite,oke,pri,rit,spr,zer
0,0.433708,0.433708,0.558478,0.0,0.000000,0.0,0.000000,0.558478,0.000000,0.000000,0.000000,0.000000
1,0.228215,0.228215,0.000000,0.0,0.386401,0.0,0.386401,0.000000,0.386401,0.386401,0.386401,0.386401


In [14]:
matrix_to

,.5L,1.5,cok,die,ero,iet,ite,oke,pri,rit,spr,zer
0,0.300832,0.300832,0.387376,0.509353,0.0,0.509353,0.0,0.387376,0.0,0.0,0.0,0.0


In [15]:
# We calculate the similarity between the pairs of vectors
cosine_similarity(matrix_from, matrix_to)

array([[0.69362794],
       [0.13730861]])

In [16]:
# Alternatively we could have avoided importing the cosine similarity function
# and use the dot product.
np.dot(np.array(matrix_from.iloc[0]),
       np.array(matrix_to.iloc[0])),np.dot(np.array(matrix_from.iloc[1]),
                                           np.array(matrix_to.iloc[0]))

(0.6936279421797706, 0.1373086119752811)

### 5. Additional useful measures: Token set ratio and Levenshtein distance

In [17]:
# We create 2 strings
s1="diet coke 600ml"
s2="diet cooke diet 600ml"

# We preprocess and tokenize the strings
tokens1 = set(fuzz.utils.full_process(s1).split())
tokens2 = set(fuzz.utils.full_process(s2).split())

# We obtain the intersection and differences of both sets
intersection = tokens1.intersection(tokens2)
diff1to2 = tokens1.difference(tokens2)
diff2to1 = tokens2.difference(tokens1)

# We concatenate the ordered the sets
sorted_sect = " ".join(sorted(intersection))
sorted_1to2 = " ".join(sorted(diff1to2))
sorted_2to1 = " ".join(sorted(diff2to1))

# We combine the results
combined_1to2 = sorted_sect + " " + sorted_1to2
combined_2to1 = sorted_sect + " " + sorted_2to1

# We remove redundant whitespaces
sorted_sect = sorted_sect.strip()
combined_1to2 = combined_1to2.strip()
combined_2to1 = combined_2to1.strip()

In [18]:
tokens1

{'600ml', 'coke', 'diet'}

In [19]:
# We calculate the fuzz ratio
pairwise = [fuzz.ratio(sorted_sect, combined_1to2)/100,
            fuzz.ratio(sorted_sect, combined_2to1)/100,
            fuzz.ratio(combined_1to2, combined_2to1)/100]
pairwise

[0.8, 0.77, 0.97]

In [20]:
# We pick the max
max(pairwise)

0.97

In [21]:
# which is obtained by comparing
combined_1to2, combined_2to1

('600ml diet coke', '600ml diet cooke')

In [22]:
# bonus, levenshtein distance:
edit_distance(combined_1to2,combined_2to1)

1

### 6. Building a baseline model

In [23]:
# Call the TFIDF vectorizer from PolyFuzz while specifying:
 # the n_grams in which the list items will be partitioned,
 # the min similarity to be considered in the output
 # whether or not stopwords and punctuations will be cleared

tfidf_vectorizer = TFIDF(n_gram_range=(3,3),clean_string=True)

# Define an instance of the PolyFuzz class using the tfidf_vectorizer model
model = PolyFuzz(tfidf_vectorizer)

# Specify a list from which its items will be matched to a refrence list
from_data = list(external_data["product_cleaned_from"])
to_data = list(internal_data["product_cleaned_to"])

# Send the lists of elements for the matching
model.match(from_data, to_data)

# Obtain the matches
matches = model.get_matches()
matches.dropna(inplace=True)

# Drop duplicates
data_matched = matches.drop_duplicates(["From","To"]).reset_index(drop=True).copy()

# Add some additional similarity measures
 # Token set ratio 
data_matched["Token_set_ratio"]=data_matched.apply(lambda x: fuzz.token_set_ratio(x["From"],x["To"])/100,axis=1)

 # Levenshtein distance
data_matched["Edit_distance"]= data_matched.apply(lambda x: edit_distance(x['From'],x['To']),axis=1)

# Extract the unity of measures of the products
data_matched["Quantity_from"]=data_matched["From"].str.extract(r"(\d+?\.?\d+?\s?ml$|\d?\d+?\.?\d+?\s?l$|\s\d?l$)")
data_matched["Quantity_to"]=data_matched["To"].str.extract(r"(\d+?\.?\d+?\s?ml$|\d?\d+?\.?\d+?\s?l$|\s\d?l$)")

# Specify the origin columns used for the matching process to be able to match the dataframes
data_matched["Property_from"] = "product_cleaned_from"
data_matched["Property_to"] = "product_cleaned_to"

# Match the results with the input data to associate them with all their data
# First we merge the external_data
data_matched = pd.merge(data_matched,external_data,left_on="From",right_on=f"{data_matched.Property_from[0]}")

# Then we use the previous join to merge it with the internal data
final_results = pd.merge(data_matched,internal_data,left_on="To",right_on="product_cleaned_to")

final_results=final_results[["product","product_name","From","To","Similarity",
                             "Token_set_ratio","Edit_distance","Quantity_from","Quantity_to"]]

In [24]:
final_results.sample(20)

,product,product_name,From,To,Similarity,Token_set_ratio,Edit_distance,Quantity_from,Quantity_to
2933,beverage diiet acid pepsi 354 ml.,DIET VANILLA PEPSI X 354ML,diiet acid pepsi 354ml,diet vanilla pepsi x 354ml,0.677,0.67,8,354ml,354ml
4229,beverage chocolate afnta 354 ml,DIET CHOCOLATE FANTA X 354ML,chocolate afnta 354ml,diet chocolate fanta x 354ml,0.708,0.83,9,354ml,354ml
2508,beverage diet acid fanta 354 ml,DIET VANILLA FANTA X 354ML,diet acid fanta 354ml,diet vanilla fanta x 354ml,0.774,0.86,7,354ml,354ml
1215,beverage diet acid spriite 1.5 LT.,DIET VANILLA SPRITE X 1.5L,diet acid spriite 1.5l,diet vanilla sprite x 1.5l,0.489,0.67,8,1.5l,1.5l
4201,beverage diet chocolate afnta 354 ml.,DIET CHOCOLATE FANTA X 354ML,diet chocolate afnta 354ml,diet chocolate fanta x 354ml,0.772,0.93,4,354ml,354ml
955,beverage diiet vanilla spriite 500 cc,DIET VANILLA SPRITE X 500ML,diiet vanilla spriite 500ml,diet vanilla sprite x 500ml,0.730,0.93,4,500ml,500ml
2497,beverage diet vanilla fanta 354cc,DIET VANILLA FANTA X 354ML,diet vanilla fanta 354ml,diet vanilla fanta x 354ml,1.000,1.00,2,354ml,354ml
3247,beverages cokke dit chocolate 354ml,DIET CHOCOLATE COKE X 354ML,cokke dit chocolate 354ml,diet chocolate coke x 354ml,0.724,0.92,14,354ml,354ml
1066,beverages spriite dit vanilla 1.5L,DIET VANILLA SPRITE X 1.5L,spriite dit vanilla 1.5l,diet vanilla sprite x 1.5l,0.678,0.92,17,1.5l,1.5l
2833,beverage diet acid pepsi 2.25 lt,DIET VANILLA PEPSI X 2.25L,diet acid pepsi 2.25l,diet vanilla pepsi x 2.25l,0.764,0.86,7,2.25l,2.25l
